In [ ]:
# =============================================================================
# PHÂN TÍCH NGƯỜI CHƠI TENNIS - SỬ DỤNG PlayerAnalysisService
# =============================================================================
# Chạy cell này để phân tích video và nhận kết quả JSON
# Hình ảnh/video trả về dạng link: outputs/{request_id}/filename.png

import sys
import os
import json
import uuid

sys.path.insert(0, os.path.abspath("."))

from src.core.player_analysis_service import PlayerAnalysisService

# ====== CẤU HÌNH ======
VIDEO_PATH = "video_test.mp4"

# Tọa độ 12 điểm sân đã xác định
# Điểm 3 (index 2) và điểm 9 (index 8) là đường lưới
COURT_POINTS = [
    (1094, 144),
    (1066, 198),
    (1037, 235),
    (988, 317),
    (740, 642),
    (402, 437),
    (240, 318),
    (539, 181),
    (647, 137),
    (714, 117),
    (824, 79),
    (957, 104),
]

# Tạo request_id và output folder
REQUEST_ID = uuid.uuid4().hex[:12]
OUTPUT_DIR = f"outputs/{REQUEST_ID}"

print("=" * 70)
print("🎾 PHÂN TÍCH NGƯỜI CHƠI TENNIS")
print("=" * 70)
print(f"📹 Video: {VIDEO_PATH}")
print(f"📁 Output: {OUTPUT_DIR}")
print(f"🆔 Request ID: {REQUEST_ID}")

In [ ]:
# =============================================================================
# CHẠY PHÂN TÍCH VÀ TRẢ VỀ JSON
# =============================================================================

# Khởi tạo service (không cần pose_model_path - đã bỏ)
service = PlayerAnalysisService(
    ball_model_path="models/ball_best.pt",
    person_model_path="yolo11m.pt",
    batch_size=16  # Optimized for 12GB GPU
)

print("🔧 Service đã khởi tạo:")
print(f"   - Ball model: models/ball_best.pt")
print(f"   - Person model: yolo11m.pt")
print(f"   - Batch size: 16")

# Chạy phân tích
print("\n⏳ Đang phân tích video...")
result = service.analyze(
    video_path=VIDEO_PATH,
    court_points=COURT_POINTS,
    output_folder=OUTPUT_DIR,
    net_start_idx=2,      # Điểm 3
    net_end_idx=8,        # Điểm 9
    ball_conf=0.3,
    person_conf=0.3,
    angle_threshold=50,
    intersection_threshold=50,
    base_url=f"outputs/{REQUEST_ID}",  # Link dạng: outputs/abc123/file.png
    create_highlights=True  # Tạo video highlight
)

# Thêm request_id
result["request_id"] = REQUEST_ID

print("\n" + "=" * 70)
print("✅ PHÂN TÍCH HOÀN TẤT!")
print("=" * 70)

In [ ]:
# =============================================================================
# HIỂN THỊ KẾT QUẢ JSON
# =============================================================================

print("=" * 70)
print("📋 KẾT QUẢ JSON")
print("=" * 70)

# In JSON đẹp
print(json.dumps(result, indent=2, ensure_ascii=False, default=str))

In [ ]:
# =============================================================================
# LƯU KẾT QUẢ JSON RA FILE VÀ HIỂN THỊ CHI TIẾT
# =============================================================================

json_path = f"{OUTPUT_DIR}/analysis_results.json"
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2, default=str)

print(f"✅ Đã lưu JSON tại: {json_path}")

# ==== THÔNG TIN VIDEO ====
print("\n" + "=" * 70)
print("📹 THÔNG TIN VIDEO")
print("=" * 70)
video_info = result['video_info']
print(f"   Path: {video_info['path']}")
print(f"   FPS: {video_info['fps']}")
print(f"   Tổng frames: {video_info['total_frames']}")
print(f"   Thời lượng: {video_info['duration_seconds']}s")

# ==== BALL TRACKING ====
print("\n" + "=" * 70)
print("🎾 BALL TRACKING")
print("=" * 70)
ball_info = result['ball_tracking']
print(f"   Tổng frames: {ball_info['total_frames']}")
print(f"   Frames detect được: {ball_info['detected_frames']}")
print(f"   Frames bị miss (trước nội suy): {ball_info['missed_frames_before_interpolation']}")
print(f"   Frames bị miss (sau nội suy): {ball_info['missed_frames_after_interpolation']}")
print(f"   Frames đã nội suy: {ball_info['interpolated_frames']}")
print(f"   Tỷ lệ detect: {ball_info['detection_rate']}%")
print(f"   Độ phủ sau nội suy: {ball_info['coverage_after_interpolation']}%")

# ==== TÓM TẮT KẾT QUẢ ====
print("\n" + "=" * 70)
print("📊 TÓM TẮT KẾT QUẢ")
print("=" * 70)
summary = result['summary']
print(f"   🆔 Request ID: {result['request_id']}")
print(f"   👥 Số người chơi: {summary['total_players']}")
print(f"   🎾 Tổng cú đánh: {summary['total_hits']}")
print(f"      ✅ Trong sân: {summary['total_in_court']}")
print(f"      ❌ Ngoài sân: {summary['total_out_court']}")
print(f"      🚫 Không qua lưới: {summary['total_not_over_net']}")

# ==== BẢNG XẾP HẠNG ====
print("\n" + "=" * 70)
print("🏆 BẢNG XẾP HẠNG")
print("=" * 70)
for r in result['rankings']:
    print(f"   #{r['rank']} Player {r['player_id']}:")
    print(f"      - Điểm: {r['score']:.2f}")
    print(f"      - Tổng cú đánh: {r['total_hits']}")
    print(f"      - Tỷ lệ trong sân: {r['in_out_ratio']*100:.1f}%")
    print(f"      - Tốc độ TB: {r['avg_speed']:.1f} pixels/frame")

# ==== CHI TIẾT TỪNG NGƯỜI CHƠI ====
print("\n" + "=" * 70)
print("👥 CHI TIẾT TỪNG NGƯỜI CHƠI")
print("=" * 70)
for player_key, player_data in result['players'].items():
    pid = player_data['player_id']
    print(f"\n🎾 PLAYER {pid}:")
    
    # Accuracy
    acc = player_data['accuracy']
    print(f"   📈 Độ chính xác:")
    print(f"      - Tổng cú đánh: {acc['total_hits']}")
    if acc['total_hits'] > 0:
        print(f"      - Trong sân: {acc['in_court']} ({acc['in_court']/acc['total_hits']*100:.1f}%)")
    else:
        print(f"      - Trong sân: 0")
    print(f"      - Ngoài sân: {acc['out_court']}")
    print(f"      - Không qua lưới: {acc['not_over_net']}")
    
    # Serve
    serve = player_data['serve']
    print(f"   🏐 Giao bóng (Serve):")
    print(f"      - Tổng: {serve['total']}")
    print(f"      - Trong sân: {serve['in_court']}, Ngoài sân: {serve['out_court']}, Không qua lưới: {serve['not_over_net']}")
    print(f"      - Tốc độ TB: {serve['avg_speed']} px/f, Max: {serve['max_speed']} px/f")
    
    # Return
    ret = player_data['return']
    print(f"   🔄 Trả bóng (Return):")
    print(f"      - Tổng: {ret['total']}")
    print(f"      - Trong sân: {ret['in_court']}, Ngoài sân: {ret['out_court']}, Không qua lưới: {ret['not_over_net']}")
    
    # Drive
    drive = player_data['drive']
    print(f"   💨 Drive:")
    print(f"      - Tốc độ TB: {drive['avg_speed']} px/f, Max: {drive['max_speed']} px/f")
    
    # Shot density
    density = player_data['shot_density']
    print(f"   📊 Mật độ bóng:")
    print(f"      - Dài: {density['long_pct']}%, Vừa: {density['medium_pct']}%, Ngắn: {density['short_pct']}%")
    
    # Highlight info
    if player_data.get('highlight'):
        hl = player_data['highlight']
        print(f"   🎬 Video Highlight:")
        print(f"      - Path: {hl.get('video_path', 'N/A')}")
        print(f"      - Số cú đánh: {hl.get('hit_count', 0)}")
        print(f"      - Thời lượng: {hl.get('duration_seconds', 0):.1f}s")

# ==== PERFORMANCE ====
print("\n" + "=" * 70)
print("⚡ HIỆU SUẤT XỬ LÝ")
print("=" * 70)
perf = result['performance']
print(f"   Tổng thời gian: {perf['total_time_seconds']}s")
print(f"   FPS xử lý: {perf['average_fps']}")
print(f"   Batch size: {perf['batch_size']}")
print(f"\n   Chi tiết thời gian:")
for step, time_sec in perf['timings'].items():
    print(f"      - {step}: {time_sec}s")

# ==== CÁC FILE ĐÃ TẠO ====
print("\n" + "=" * 70)
print("📁 CÁC FILE ĐÃ TẠO")
print("=" * 70)
vis = result['visualizations']
if vis.get('combined_highlight'):
    print(f"   🎬 Combined highlight: {vis['combined_highlight']}")

print("\n   📊 Per-player files:")
for pid, files in vis['per_player'].items():
    print(f"      Player {pid}:")
    for name, path in files.items():
        print(f"         - {name}: {path}")

In [ ]:
# =============================================================================
# HIỂN THỊ HÌNH ẢNH VÀ VIDEO (CHẠY TRONG JUPYTER)
# =============================================================================

from IPython.display import Image, display, Video, HTML
import glob

print("=" * 70)
print("🖼️ HIỂN THỊ HÌNH ẢNH")
print("=" * 70)

# Hiển thị heatmap từng người
heatmap_files = glob.glob(f"{OUTPUT_DIR}/player_*_heatmap.png")
for hf in sorted(heatmap_files):
    print(f"\n🗺️ {os.path.basename(hf)}:")
    display(Image(filename=hf))

# Hiển thị player images
player_images = glob.glob(f"{OUTPUT_DIR}/player_*_image.jpg")
for pi in sorted(player_images):
    print(f"\n👤 {os.path.basename(pi)}:")
    display(Image(filename=pi))

# ==== VIDEO HIGHLIGHTS ====
print("\n" + "=" * 70)
print("🎬 VIDEO HIGHLIGHTS")
print("=" * 70)

# Combined highlight
combined_path = f"{OUTPUT_DIR}/combined_highlight.mp4"
if os.path.exists(combined_path):
    print("\n🎬 Combined Highlight Video:")
    print(f"   Path: {combined_path}")
    try:
        display(Video(combined_path, embed=True, width=640))
    except:
        print(f"   (Mở file để xem: {combined_path})")

# Per-player highlights
highlight_files = glob.glob(f"{OUTPUT_DIR}/player_*_highlight.mp4")
for hv in sorted(highlight_files):
    print(f"\n🎬 {os.path.basename(hv)}:")
    try:
        display(Video(hv, embed=True, width=640))
    except:
        print(f"   (Mở file để xem: {hv})")

In [1]:
import cv2

# ====== CONFIG ======
VIDEO_PATH = "video_test.mp4"
POINT_RADIUS = 5
POINT_COLOR = (0, 0, 255)      # đỏ
LINE_COLOR = (255, 0, 0)       # xanh
LINE_THICKNESS = 2
# ====================

points = []
paused = False
frame_copy = None


def mouse_callback(event, x, y, flags, param):
    global points, frame_copy

    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))
        redraw()


def redraw():
    global frame_copy
    frame_copy[:] = frame_original.copy()

    # vẽ point
    for p in points:
        cv2.circle(frame_copy, p, POINT_RADIUS, POINT_COLOR, -1)

    # nối point
    for i in range(1, len(points)):
        cv2.line(
            frame_copy,
            points[i - 1],
            points[i],
            LINE_COLOR,
            LINE_THICKNESS
        )


cap = cv2.VideoCapture(VIDEO_PATH)
cv2.namedWindow("Video")
cv2.setMouseCallback("Video", mouse_callback)

while cap.isOpened():
    if not paused:
        ret, frame = cap.read()
        if not ret:
            break

        frame_original = frame.copy()
        frame_copy = frame.copy()

    cv2.imshow("Video", frame_copy)

    key = cv2.waitKey(30) & 0xFF

    if key == ord('q'):
        break

    elif key == ord(' '):   # pause / play
        paused = not paused

    elif key == ord('r'):   # reset points
        points.clear()
        redraw()

    elif key == ord('u'):   # undo point
        if points:
            points.pop()
            redraw()

    elif key == ord('s'):   # save image
        cv2.imwrite("keypoints_result.png", frame_copy)
        print("Saved keypoints_result.png")

cap.release()
cv2.destroyAllWindows()
